<a href="https://colab.research.google.com/github/ujin2021/tensorflow-Tutorial/blob/master/model_tips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 학습이 잘 되는 모델
* 사용할 레이어
  * tf.keras.layers.BatchNormaliztion()
  * tf.keras.layers.Activation('swish')

In [21]:
# 라이브러리 사용
import tensorflow as tf
import pandas as pd

### Boston 집값 예측

In [16]:
# 1. 과거의 data 준비
path = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/boston.csv'
boston = pd.read_csv(path)

# 종속변수, 독립변수
idp = boston[['crim', 'zn', 'indus', 'chas', 'nox', 
            'rm', 'age', 'dis', 'rad', 'tax',
            'ptratio', 'b', 'lstat']]
dp = boston[['medv']]
print(idp.shape, dp.shape)

(506, 13) (506, 1)


In [4]:
# # 2. 모델의 구조를 만든다(기존의 방법)
# X = tf.keras.layers.Input(shape=[13])
# # 각 node가 8개인 hidden layer 3개
# H = tf.keras.layers.Dense(8, activation='swish')(X)
# H = tf.keras.layers.Dense(8, activation='swish')(H)
# H = tf.keras.layers.Dense(8, activation='swish')(H)
# Y = tf.keras.layers.Dense(1)(H)
# model = tf.keras.models.Model(X, Y)
# model.compile(loss='mse')
# model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 112       
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 265
Trainable params: 265
Non-trainable params: 0
_________________________________________________________________


In [17]:
# 2. 모델의 구조를 만든다
X = tf.keras.layers.Input(shape=[13])

# 위의 H = tf.keras.layers.Dense(8, activation='swish')(H) 를 분리, 그사이에 Batch 넣음
H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H) # 사이에 넣는게 효과가 있다
H = tf.keras.layers.Activation('swish')(H)
H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)
H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='mse')
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 13)]              0         
_________________________________________________________________
dense_20 (Dense)             (None, 8)                 112       
_________________________________________________________________
batch_normalization_9 (Batch (None, 8)                 32        
_________________________________________________________________
activation_9 (Activation)    (None, 8)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 8)                 72        
_________________________________________________________________
batch_normalization_10 (Batc (None, 8)                 32        
_________________________________________________________________
activation_10 (Activation)   (None, 8)               

In [18]:
# 3. data로 model 학습하기
model.fit(idp, dp, epochs=1000, verbose=0)
model.fit(idp, dp, epochs=10)
# 그러나 loss가 떨어지는 양이 크지 않다. -> 어떻게 해결?

Epoch 1/10
16/16 [==============================] - 0s 2ms/step - loss: 8.6445
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 12.4512
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 11.6053
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 10.9332
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 13.6467
Epoch 6/10
16/16 [==============================] - 0s 2ms/step - loss: 12.8255
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 10.7326
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 11.1759
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 10.1482
Epoch 10/10
16/16 [==============================] - 0s 2ms/step - loss: 13.9518


### iris 품종 예측

In [19]:
# 1. 과거의 데이터 준비
path = 'https://raw.githubusercontent.com/blackdew/tensorflow1/master/csv/iris.csv'
iris = pd.read_csv(path)

# one hot encoding
iris = pd.get_dummies(iris)

# 종속변수, 독립변수
iris_idp = iris[['꽃잎길이', '꽃잎폭', '꽃받침길이', '꽃받침폭']]
iris_dp = iris[['품종_setosa', '품종_versicolor', '품종_virginica']]
print(iris_idp.shape, iris_dp.shape)

(150, 4) (150, 3)


In [23]:
# 2. 모델의 구조를 만든다
X = tf.keras.layers.Input(shape=[4])

H = tf.keras.layers.Dense(8)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

H = tf.keras.layers.Dense(8)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('swish')(H)

Y = tf.keras.layers.Dense(3, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.compile(loss='categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_32 (Dense)             (None, 8)                 40        
_________________________________________________________________
batch_normalization_18 (Batc (None, 8)                 32        
_________________________________________________________________
activation_18 (Activation)   (None, 8)                 0         
_________________________________________________________________
dense_33 (Dense)             (None, 8)                 72        
_________________________________________________________________
batch_normalization_19 (Batc (None, 8)                 32        
_________________________________________________________________
activation_19 (Activation)   (None, 8)               

In [29]:
# 3. 데이터로 모델을 학습
model.fit(iris_idp, iris_dp, epochs=1000, verbose=0)
model.fit(iris_idp, iris_dp, epochs=10)

Epoch 1/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0327 - accuracy: 0.9933
Epoch 2/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0247 - accuracy: 0.9933
Epoch 3/10
5/5 [==============================] - 0s 3ms/step - loss: 0.0415 - accuracy: 0.9933
Epoch 4/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0291 - accuracy: 0.9867
Epoch 6/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0079 - accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0326 - accuracy: 0.9933
Epoch 8/10
5/5 [==============================] - 0s 3ms/step - loss: 0.0142 - accuracy: 0.9933
Epoch 9/10
5/5 [==============================] - 0s 3ms/step - loss: 0.0648 - accuracy: 0.9667
Epoch 10/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0192 - accuracy: 0.9933
